<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# **SQL Server 2019 big data cluster Tutorial**
## **04 - Data Virtualization**

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster.  
- You'll start with creating a text file format, since that's the type of data you are reading in. 
- Next, you'll create a data source for the SQL Storage Pool, since that allows you to access the HDFS system in BDC. 
- Finally, you'll create an External Table, which uses the previous steps to access the data.


In [ ]:
USE WideWorldImporters;
SELECT * FROM sys.external_tables

In [ ]:
/* Clean up only - run this cell only if you are repeating the tutorial! */
USE WideWorldImporters;
GO

IF EXISTS(SELECT * FROM sys.external_tables WHERE name = 'partner_customers_hdfs')
BEGIN
    DROP EXTERNAL TABLE partner_customers_hdfs
END

IF EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
    DROP EXTERNAL DATA SOURCE SqlStoragePool
END

IF EXISTS(SELECT * FROM sys.external_file_formats WHERE name = 'csv_file')
BEGIN
    DROP EXTERNAL FILE FORMAT csv_file
END

In [ ]:
/* Create External File Format */
USE WideWorldImporters;
GO

CREATE EXTERNAL FILE FORMAT csv_file
WITH (
    FORMAT_TYPE = DELIMITEDTEXT,
    FORMAT_OPTIONS(
        FIELD_TERMINATOR = ',',
        STRING_DELIMITER = '0x22',
        FIRST_ROW = 2,
        USE_TYPE_DEFAULT = TRUE)
);
GO

In [ ]:
/* Create External Data Source to the Storage Pool */
CREATE EXTERNAL DATA SOURCE SqlStoragePool
WITH (LOCATION = 'sqlhdfs://controller-svc/default');

In [ ]:
/* Create an External Table that can read from the Storage Pool File Location */
CREATE EXTERNAL TABLE [partner_customers_hdfs]
    ("CustomerSource" VARCHAR(250) 
    , "CustomerName" VARCHAR(250) 
    , "EmailAddress" VARCHAR(250))
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/partner_customers',
        FILE_FORMAT = csv_file
    );
GO

In [ ]:
/* Read Data from HDFS using only T-SQL */
SELECT TOP 10 CustomerSource
, CustomerName
, EMailAddress
    FROM [partner_customers_hdfs] hdfs
WHERE EmailAddress LIKE '%wingtip%'
ORDER BY CustomerSource, CustomerName;
GO


In [ ]:
/* Now Join Those to show customers we currently have in a SQL Server Database 
and the Category they qre in the External Table */
USE WideWorldImporters;
GO

SELECT TOP 10 a.FullName
  , b.CustomerSource
  FROM Application.People a
  INNER JOIN partner_customers_hdfs b  ON a.FullName = b.CustomerName
  ORDER BY FullName ASC;
  GO

## **Next Steps: Continue on to Working with the SQL Server Data Pool**

Now you're ready to open the next Jupyter Notebook - [bdc-05-data-mart.ipynb](bdc-05-data-mart.ipynb) - to learn how to create and work with a Data Mart.